# Routing Topology: OSPF using FRRouting

This notebook is an example of how to create a FABRIC routing experiment topology comprising nodes at three different sites. Each site has a local layer 2 (Ethernet) network connecting a set of local nodes and one gateway router. The three gateway routers connect to each other and use the [FRRouting](https://frrouting.org/) protocol suite to deploy [OSPF](https://en.wikipedia.org/wiki/Open_Shortest_Path_First) dameons to propagate route updates across the topology.

You might be familiar with the [Quagga](https://www.quagga.net/) router suite.  FRRouting is based on Quagga but has a more active upstream community including many large companies working on cloud networking.


<img src="./figs/frr.png" width="90%"><br>


## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();


## Create the Experiment Slice

The following creates private layer 2 networks on three sites including OSPF gateway routers that propagate routes across the topology. 


In [ ]:
slice = fablib.new_slice(name="frrouting")
slice.load('frrouting.graphml')
slice.submit()


## Run the Experiment

We will just test `ping` RTT and look at `tracepath`. Your experiment should be more interesting!

Notice that if you run this quickly and repeatedly run this test against a specific target, you may see changes to the tracepath.  Initially the ping may even fail.  Why do you think this is happening?


In [ ]:
src_node_name =  f'ucsd1'
dst_node_name =  f'wash1'
dst_net_name ='net_localWASH'

slice = fablib.get_slice(name="frrouting")

src_node = slice.get_node(name=src_node_name)
dst_node = slice.get_node(name=dst_node_name)

target_ip=dst_node.get_interface(network_name=dst_net_name).get_ip_addr()
stdout, stderr = src_node.execute(f'ping -c 5 {target_ip}')


## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name='frrouting')
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")